In [ ]:
import yfinance as yf
import numpy as np

ticker = 'TSLA'

start_date = "2015-01-01"
df = yf.download(
    ticker, 
    interval='1d',
    # start=start_date,
    period='10y'
)
df.columns = df.columns.get_level_values(0)
df = df[['Close']] # keep only close prices
df['return'] = df['Close'].pct_change()
df['state'] = np.where(df['return'] >= 0, 0, 1)

In [ ]:
df.to_csv('data.csv')

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('data.csv', index_col = 'Date')

In [2]:
def getReturnVolatility(df, state=0, period=50): 
    if state not in df['state'].unique():
        print('Invalid state')
        return -1
    local_df = df.copy()
    local_df['segment'] = pd.qcut(range(len(local_df)), q=period, labels=False)

    volatility_per_segment = local_df[local_df['state'] == state].groupby('segment')['return'].var()
    weight = pd.Series([2**(-(period-i)) for i in range(period)])
    return np.sqrt(volatility_per_segment.dot(weight) * 252) # convert to annual vol

In [3]:
vols = [ getReturnVolatility(df, i, 100) for i in range(2) ]
vols

[np.float64(0.14669682762693306), np.float64(0.1983869414748875)]

### Phase 2: building matrices

In [6]:
import numpy as np
import pandas as pd
from functools import cache

In [11]:
# generating matrix A
n = .5
A = np.matrix([[-n, n], [n, -n]])

T = 1 # years
N = 1000
delta_t = T/N
def get_markov_transition_prob(delta_t, A, max_iter=50):
    res = np.eye(A.shape[0])
    prev = np.eye(A.shape[0])
    for i in range(max_iter):
        prev = prev @ A * delta_t / (i+1)
        res += prev
    return res

P = get_markov_transition_prob(delta_t, A)

# option pricing params
# vols = [.25, .35]
sigma = max(vols) + (np.sqrt(1.5)-1) * np.mean(vols) # suggested by the paper
# S0 = 100
S0 = df['Close'].loc['2025-01-02'] # take a close price
risk_free_int = [.04, .04] # adjustable, or make them stochastic

risk_neutral_prob = { i: [-1]*3 for i in range(P.shape[0]) } # [up, mid, down]
for i in range(P.shape[0]):
    risk_neutral_prob[i][1] = 1 - (vols[i]/sigma) ** 2
    risk_neutral_prob[i][0] = (np.exp(risk_free_int[i]*delta_t)- np.exp(-sigma * np.sqrt(delta_t))- \
                               (1-(vols[i]/sigma)**2)*(1-np.exp(-sigma * np.sqrt(delta_t))))/(np.exp(sigma * np.sqrt(delta_t)) - np.exp(-sigma * np.sqrt(delta_t)))
    risk_neutral_prob[i][2] = (np.exp(sigma * np.sqrt(delta_t)) - np.exp(risk_free_int[i]*delta_t)- \
                               (1-(vols[i]/sigma)**2)*(np.exp(sigma * np.sqrt(delta_t))-1))/(np.exp(sigma * np.sqrt(delta_t)) - np.exp(-sigma * np.sqrt(delta_t)))


def option_value(time, node, state, K):
    """
    V(t,n,j) be the value of the derivative at the nth node at time step t under the jth regime state.
    time starts from 0
    node index starts from 1
    """
    if state >= P.shape[0]:
        raise Exception('Invalid state')

    assert node <= 2 * time + 1 
    
    @cache
    def V(t, n, i):
        if t == N:
            return max(0, S0*np.exp((n - 1 - N)*sigma*np.sqrt(delta_t))-K)
        res = 0
        for j in range(P.shape[0]): # regimes
            res += P[i][j] * (risk_neutral_prob[i][0] * V(t+1, n+2, j) + risk_neutral_prob[i][1] * V(t+1, n+1, j) + risk_neutral_prob[i][2] * V(t+1, n, j))
        return np.exp(-risk_free_int[i]*delta_t) * res
    
    return V(time, node, state)
    


In [12]:
strike = 250
print(f'Initial price: {S0:.2f}')
print('vols=', np.round(vols, 4))
print(f'Option price: {option_value(0, 1, 1, strike):.3f}')

Initial price: 243.58
vols= [0.1467 0.1984]
Option price: 19.993
